In [ ]:
import requests


url = 'http://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz'
r = requests.get(url, allow_redirects=True)

open('gutenberg-poetry-v001.ndjson.gz', 'wb').write(r.content)

54837372

In [ ]:
!gunzip gutenberg-poetry-v001.ndjson.gz

In [ ]:
!sudo apt-get install jq

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libjq1 libonig5
The following NEW packages will be installed:
  jq libjq1 libonig5
0 upgraded, 3 newly installed, 0 to remove and 21 not upgraded.
Need to get 313 kB of archives.
After this operation, 1,062 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libonig5 amd64 6.9.4-1 [142 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libjq1 amd64 1.6-1ubuntu0.20.04.1 [121 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 jq amd64 1.6-1ubuntu0.20.04.1 [50.2 kB]
Fetched 313 kB in 2s (196 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog ba

In [ ]:
import json
import os
import sys

def split_by_book(poems):
    prev_gid = -1
    r = []
    for entry in poems:
        gid = entry['gid']
        if gid != prev_gid:
            if prev_gid != -1 and len(r) > 0:
                yield r
                r = []
            prev_gid = gid
        r += [entry]
    if len(r) > 0:
        yield r

def split_by_transition(poems):
    prev_line = -2
    r = []
    for entry in poems:
        line = prev_line + 1
        if line != prev_line + 1:
            if prev_line != -2 and len(r) > 0:
                yield r
                r = []
            prev_line = line
        r += [entry]
    if len(r) > 0:
        yield r

def extract(entries):
    return '\n'.join(list(map(lambda x: x['s'], entries)))

def final(poems):
    for book in split_by_book(poems):
        yield ''
        for stanza in split_by_transition(book):
            if len(stanza) > 5:
                yield '\n'
                for entry in stanza:
                    line = entry['s']
                    line = line.rstrip('} ')
                    yield line

def output(poems, fname):
    with open(fname, "w") as f:
        for line in final(poems):
            f.write(line + '\n')

def main():
    print('Loading poems...')
    with open("gutenberg-poetry-v001.ndjson") as f:
        poems=[json.loads(line) for line in f]
    print('Loaded.')
    output(poems, "corpus.txt")

if __name__ == "__main__":
    main()


Loading poems...
Loaded.


In [ ]:
!pip install tokenizer
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 KB 6.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
from transformers import WEIGHTS_NAME, CONFIG_NAME
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>')


In [ ]:
textfile = open("corpus.txt", "r", encoding='utf-8')
text = textfile.read()
textfile.close()

In [ ]:
string_tokenized = tokenizer.encode(text)
print("Done tokenizing")

In [ ]:
examples = []
block_size = 200
BATCH_SIZE = 4
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
    examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []

In [ ]:
for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print("Done creating dataset")

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
# Exponential decay learning rate
  optimizer = tf.keras.optimizers.Adam(
    learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  configuration = GPT2Config.from_pretrained(
    pretrained_model_name_or_path  = "gpt2-medium",
    vocab_size=tokenizer.vocab_size,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
    )
  model = TFGPT2LMHeadModel(config=configuration)
  model = model.from_pretrained("/content/drive/MyDrive/models5")
  model.resize_token_embeddings(len(tokenizer))
  model.compile(optimizer=optimizer, loss=loss)
model.summary()

In [ ]:
model.fit(dataset, epochs=2)

In [ ]:
save_location = "/content/drive/MyDrive/models"
if not os.path.exists(save_location):
    os.makedirs(save_location)
model.save_pretrained(save_location)
tokenizer.save_pretrained(save_location)

In [ ]:
text = "my mistress's eyes are nothing like the sun"
input_ids = tokenizer.encode(text, return_tensors='tf')

beam_output = model.generate(
     input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.9,
    temperature=1.0,
    num_return_sequences=2,
    no_repeat_ngram_size=2
)

print(tokenizer.decode(beam_output[1], skip_special_tokens=True))

In [ ]:
configuration = GPT2Config.from_pretrained(
    pretrained_model_name_or_path  = "gpt2-medium",
    vocab_size = tokenizer.vocab_size,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
    )
model = TFGPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path  = "gpt2-medium", config=configuration)
model = model.from_pretrained("/content/drive/MyDrive/models2")
tokenizer = tokenizer.from_pretrained("/content/drive/MyDrive/models2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/models.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
text = "The dreamless night"
input_ids = tokenizer.encode(text, return_tensors='tf')

beam_output = model.generate(
     input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.9,
    temperature=1.0,
    num_return_sequences=2,
    no_repeat_ngram_size=2
)

print(tokenizer.decode(beam_output[1], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The dreamless night.
The dark-eyed waves grew cold,
Breaths, on a bird's eyes;
No night is no longer more. The child
Is I with her head, the sun. There came,--



In [ ]:
save_location = "/content/drive/MyDrive/models"
if not os.path.exists(save_location):
    os.makedirs(save_location)
model.save_pretrained(save_location)
tokenizer.save_pretrained(save_location)